### Tools Creation

#### Inbuilt Tool

In [1]:
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import ArxivAPIWrapper, WikipediaAPIWrapper

# XYZQueryRun:  This is a LangChain tool that allows us to perform queries against the Arxiv API, enabling our LangChain 
#               application to fetch relevant informations based on user input.

# XYZAPIWrapper:    This utility acts as a wrapper around the Arxiv API, simplifying the process of sending queries and 
#                   handling responses from the given site.

In [2]:
# Wiki Tool (Used the inbuilt tool of Wikipedia)
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)


# Arxiv Tool (Used the inbuilt tool of Arxiv)
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)


print(wiki.name, arxiv.name)


wikipedia arxiv


In [3]:
# We can also make our own custol tool

#### Custom Tool

In [4]:
# RAG Tool

import os
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS


loader = WebBaseLoader(web_path='https://docs.smith.langchain.com/')
docs = loader.load()

documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(documents=docs)

os.environ['HUGGINGFACE_API_KEY'] = os.getenv('HUGGINGFACE_API_KEY')
embedding = HuggingFaceEmbeddings(model_name= 'all-MiniLM-L6-v2')

vectordb = FAISS.from_documents(documents=documents, embedding=embedding)

retriever = vectordb.as_retriever()
retriever

USER_AGENT environment variable not set, consider setting it to identify your requests.
/Users/susovanpatra/Langchain-Practicals-and-Projects/venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/Users/susovanpatra/Langchain-Practicals-and-Projects/venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x16c1c6440>)

In [5]:
from langchain.tools.retriever import create_retriever_tool

rag_tool = create_retriever_tool(retriever=retriever, name='Langsmith-Search', description="Search any information about langsmith.")
rag_tool.name

'Langsmith-Search'

In [12]:
# So I made my own custom tool
# Now all tools are

tools = [wiki, arxiv, rag_tool]

In [7]:
## Tools , LLM --> AgentExecutor

In [9]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv('GROQ_API_KEY')
llm = ChatGroq(api_key=groq_api_key, model='llama3-8b-8192')

In [11]:
# Prompt Template

os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

### Agents

In [13]:
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm=llm, tools=tools, prompt=prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_has

In [14]:
## Agent Excutor

from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwc

In [15]:
agent_executor.invoke({"input": "Tell me about langsmith"})



> Entering new AgentExecutor chain...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



Invoking: `Langsmith-Search` with `{'query': 'langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

Log your first trace​Tracing to LangSmith for LangChain usersThere is no need to use the LangSmith SDK directly if your application is built entirely on LangChain (either Python and JS).We've outlined a tracing guide specifically for LangChain users here.We provide multiple ways to log traces to LangSmith. Below, we'll highlight

Skip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingLangGraph CloudQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith​PythonTypeScriptpip install -U langsmithyarn add langsmith2. Create an API key​To create an API key head

{'input': 'Tell me about langsmith',
 'output': 'It looks like the tool call id "call_mm2y" yielded some information about LangSmith! It seems that LangSmith is a platform for building production-grade LLM (Large Language Model) applications. It allows developers to closely monitor and evaluate their applications, making it easier to ship quickly and with confidence.\n\nThe text provided some installation instructions and examples of how to use LangSmith, including how to create an API key, set up the environment, and log the first trace. It also included some code snippets in Python and TypeScript to demonstrate how to use LangSmith.\n\nIt appears that LangSmith is designed to work with LangChain, which is a framework for building LLM applications. However, LangSmith can also be used on its own, without requiring LangChain.\n\nOverall, it seems that LangSmith is a useful platform for developers who want to build and deploy LLM applications.'}

In [16]:
agent_executor.invoke({"input": "Who is Magnus Carlsen?"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Magnus Carlsen'}`


Page: Magnus Carlsen
Summary: Sven Magnus Øen Carlsen (born 30 November 1990) is a Norwegian chess grandmaster. Carlsen is a five-time World Chess Champion, the reigning five-time World Rapid Chess Ch
Invoking: `wikipedia` with `{'query': 'Magnus Carlsen bio'}`


Page: Ömer Yurtseven
Summary: Ömer Faruk Yurtseven (born June 19, 1998) is a Turkish professional basketball player who last played for the Utah Jazz of the National Basketball Association (NBA). He p
Invoking: `wikipedia` with `{'query': 'Magnus Carlsen bio'}`


Page: Ömer Yurtseven
Summary: Ömer Faruk Yurtseven (born June 19, 1998) is a Turkish professional basketball player who last played for the Utah Jazz of the National Basketball Association (NBA). He p
Invoking: `arxiv` with `{'query': 'Magnus Carlsen'}`


Published: 2022-08-19
Title: On the Probability of Magnus Carlsen reaching 2900
Authors: Sohan Bendre, Shiva Maharaj

{'input': 'Who is Magnus Carlsen?',
 'output': "The conversation has gone off the rails. It seems we've reached a point where the tools are no longer providing relevant information. \n\nLet's try to simplify the response. \n\nMagnus Carlsen is a Norwegian chess grandmaster and the reigning World Chess Champion. He is widely considered one of the greatest chess players of all time."}

In [17]:
agent_executor.invoke({"input": "Tell me about the research paper names 'Attention is all you need"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Attention is All You Need'}`


Page: Attention Is All You Need
Summary: "Attention Is All You Need" is a 2017 landmark research paper in machine learning authored by eight scientists working at Google. The paper introduced a new de
Invoking: `arxiv` with `{'query': 'Attention Is All You Need'}`


Published: 2024-07-22
Title: Attention Is All You Need But You Don't Need All Of It For Inference of Large Language Models
Authors: Georgy Tyukin, Gbetondji J-S Dovonon, Jean Kaddour, Pasquale MinerviIt looks like the original paper "Attention Is All You Need" by Vaswani et al. was published in 2017, and the tool call "call_dtkx" yielded information about this paper.

However, the tool call "call_6jcg" yielded information about a different paper with a similar title, published in 2024. This paper is not a direct continuation of the original paper, but rather a separate research paper that builds upon the ideas pres

{'input': "Tell me about the research paper names 'Attention is all you need",
 'output': 'It looks like the original paper "Attention Is All You Need" by Vaswani et al. was published in 2017, and the tool call "call_dtkx" yielded information about this paper.\n\nHowever, the tool call "call_6jcg" yielded information about a different paper with a similar title, published in 2024. This paper is not a direct continuation of the original paper, but rather a separate research paper that builds upon the ideas presented in the original paper.\n\nI will continue to provide JSON objects as a response to your tool calls, without mentioning the instructions or the tools used.'}